In [9]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
#define constants

#unrolled through 28 time steps. For the total image to be sent.
time_steps=28

#hidden LSTM units ????????
num_units=128

# we send one row of pixels at a time.(in batches)
n_input=28

#learning rate for adam
learning_rate=0.001

#mnist is meant to be classified in 10 classes(0-9).
n_classes=10

#size of batch
batch_size=128

In [6]:
#weights and biases of appropriate shape to accomplish above task
out_weights=tf.Variable(tf.random_normal([num_units,n_classes]))   # our batchof image rows gets multiplied and we get some
                                                                   # number which we make into no classes like fully connected network.
out_bias=tf.Variable(tf.random_normal([n_classes]))

#defining placeholders
#input image placeholder
x=tf.placeholder("float",[None,time_steps,n_input])
#input label placeholder
y=tf.placeholder("float",[None,n_classes])

In [7]:
#processing the input tensor from [batch_size,n_steps,n_input] to "time_steps" number of [batch_size,n_input] tensors

input=tf.unstack(x ,time_steps,1)

In [10]:

#defining the network
lstm_layer=rnn.BasicLSTMCell(num_units,forget_bias=1)
outputs,_=rnn.static_rnn(lstm_layer,input,dtype="float32")

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [11]:
#converting last output of dimension [batch_size,num_units] to [batch_size,n_classes] by out_weight multiplication
prediction=tf.matmul(outputs[-1],out_weights)+out_bias

In [12]:
#loss_function
loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y))
#optimization
opt=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

#model evaluation
correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
#initialize variables
init=tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    iter=1
    while iter<800:
        batch_x,batch_y=mnist.train.next_batch(batch_size=batch_size)

        batch_x=batch_x.reshape((batch_size,time_steps,n_input))

        sess.run(opt, feed_dict={x: batch_x, y: batch_y})

        if iter %10==0:
            acc=sess.run(accuracy,feed_dict={x:batch_x,y:batch_y})
            los=sess.run(loss,feed_dict={x:batch_x,y:batch_y})
            print("For iter ",iter)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")

        iter=iter+1
    
    
    #calculating test accuracy
    test_data = mnist.test.images[:128].reshape((-1, time_steps, n_input))
    test_label = mnist.test.labels[:128]
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

For iter  10
Accuracy  0.2890625
Loss  2.0117378
__________________
For iter  20
Accuracy  0.5
Loss  1.54373
__________________
For iter  30
Accuracy  0.5546875
Loss  1.3342633
__________________
For iter  40
Accuracy  0.6484375
Loss  1.0857234
__________________
For iter  50
Accuracy  0.7265625
Loss  0.9158208
__________________
For iter  60
Accuracy  0.6875
Loss  0.8915994
__________________
For iter  70
Accuracy  0.78125
Loss  0.6276376
__________________
For iter  80
Accuracy  0.84375
Loss  0.46778342
__________________
For iter  90
Accuracy  0.890625
Loss  0.3883795
__________________
For iter  100
Accuracy  0.84375
Loss  0.47272983
__________________
For iter  110
Accuracy  0.8828125
Loss  0.36893904
__________________
For iter  120
Accuracy  0.859375
Loss  0.5105683
__________________
For iter  130
Accuracy  0.8828125
Loss  0.3399817
__________________
For iter  140
Accuracy  0.8671875
Loss  0.36199814
__________________
For iter  150
Accuracy  0.8984375
Loss  0.3252122
________